In [107]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
import pyarrow.feather as feather
import time
from datetime import datetime
from datetime import timedelta


In [128]:
export_df = feather.read_feather("../data/cleaned_data.feather")


In [129]:
export_df["event_timestamp"]=pd.to_datetime(export_df["event_timestamp"]*1000, unit="ns", utc=True)

#
Create a variable for classifying notifications: 1 if dismissed/opened in time the accident is active, 0 for when it is already expired. #8

In [130]:
export_df["notif_outdated"]=list(map(lambda x,y,z: 0 if x=="notification_received" else (1 if y < z else 2), export_df["event_name"], export_df["event_timestamp"],export_df["closed_at"]))

In [132]:
export_df["reaction_time"]=list(map(lambda x,y,z,w: y-z
if (x==1 and y>z) else (y-w if (x==1 and y>w) else "Null"), 
export_df["notif_outdated"],export_df["event_timestamp"],export_df["published_at"],export_df["created_at"]))

In [134]:
export_df["reaction_time"].value_counts()

Null                      1705668
0 days 00:00:30.305001          6
0 days 00:00:21.825001          6
0 days 00:00:18.714001          6
0 days 00:04:07.070001          5
                           ...   
0 days 00:06:09.104000          1
0 days 00:04:20.433000          1
0 days 00:59:20.928000          1
0 days 01:17:13.169000          1
0 days 00:03:01.242001          1
Name: reaction_time, Length: 501600, dtype: int64

In [135]:
export_df

,event_timestamp,event_name,user_id,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,...,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_outdated,reaction_time
0,2021-05-28 22:22:07.928000+00:00,notification_received,ac6d3,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,2021-05-28 22:21:37+00:00,2021-05-28 22:21:44+00:00,2021-05-28 23:01:00+00:00,VehicularAccident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,Null
1,2021-05-28 22:29:19.952000+00:00,notification_dismiss,ac6d3,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,2021-05-28 22:21:37+00:00,2021-05-28 22:21:44+00:00,2021-05-28 23:01:00+00:00,VehicularAccident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0 days 00:07:35.952000
2,2021-05-29 00:35:53.374000+00:00,notification_received,ac6d3,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,2021-05-29 00:00:46+00:00,2021-05-29 00:21:36+00:00,2021-05-29 01:01:00+00:00,VehicularAccident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,Null
3,2021-05-29 00:36:08.285001+00:00,notification_dismiss,ac6d3,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,2021-05-29 00:00:46+00:00,2021-05-29 00:21:36+00:00,2021-05-29 01:01:00+00:00,VehicularAccident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0 days 00:14:32.285001
4,2021-05-29 03:13:31.584001+00:00,notification_received,ac6d3,-Maq63vqDUxQRawQ_3Au,YKlNxSvA6xGpKOFIXMTiIg,2021-05-29 03:12:59+00:00,2021-05-29 03:15:13+00:00,2021-05-29 13:59:03+00:00,Demonstration,CirculationShutdown,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,Null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231990,2021-07-06 14:21:02.819000+00:00,notification_received,e0b4d,-MdwBMI8mNXadCw7GwTc,kCtjX2Xe6xGa3QeEZ-Zg-g,2021-07-06 14:20:53+00:00,2021-07-06 14:20:59+00:00,2021-07-06 15:21:33+00:00,Demonstration,SuspensionOfService,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Null
2231991,2021-07-06 15:21:40.674000+00:00,notification_received,e0b4d,-MdwPCoa1U3j5wMZ8J99,YNy_023e6xGa3QeEZ-Zg-g,2021-07-06 15:21:25+00:00,2021-07-06 15:21:36+00:00,2021-07-06 20:46:48+00:00,Demonstration,SuspensionOfService,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Null
2231992,2021-07-06 16:05:21.801000+00:00,notification_received,e0b4d,-MdwZE8hMtqpcYoaNhzM,0AF_8XPe6xGa3QeEZ-Zg-g,2021-07-06 16:05:11+00:00,2021-07-06 16:05:16+00:00,2021-07-06 17:37:59+00:00,Demonstration,SuspensionOfService,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Null
2231993,2021-07-05 23:42:09.057001+00:00,notification_received,427c1,-Mdt2BT-2y05YXyJctt2,ADiylurd6xGf17VsriRlgg,2021-07-05 23:41:58+00:00,2021-07-05 23:42:01+00:00,2021-07-06 02:00:01+00:00,Rain,Delays,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Null
